In [ ]:
import pandas as pd
import json

# Загрузка данных из JSON файлов
with open('data/kyrgyz_gold.json', 'r', encoding='utf-8') as f1:
    kyrgyz_gold_data = json.load(f1)

with open('data/nbrk.json', 'r', encoding='utf-8') as f2:
    nbrk_data = json.load(f2)

# Загрузка данных из CSV файла 
optima_data = pd.read_csv('data/optima.csv')

# Преобразование данных в DataFrame
kyrgyz_gold_df = pd.DataFrame(kyrgyz_gold_data)
nbrk_df = pd.DataFrame(nbrk_data)

# Преобразование столбца 'Дата' в datetime
kyrgyz_gold_df['Дата'] = pd.to_datetime(kyrgyz_gold_df['Дата'], format='%d.%m.%Y')
nbrk_df['Дата'] = pd.to_datetime(nbrk_df['Дата'], format='%d.%m.%Y')
optima_data['Дата'] = pd.to_datetime(optima_data['Дата'])

# Функция для агрегации данных по золотым слиткам
def aggregate_gold_data(df, source):
    # Группировка по дате
    grouped = df.groupby('Дата').agg({
        'Мерные золотые слитки, г.': lambda x: ', '.join(map(str, x)),
        'Цена обратного выкупа, сом': lambda x: ', '.join(map(str, x)),
        'Цена продажи, сом': lambda x: ', '.join(map(str, x))
    }).reset_index()
    
    grouped['Источник'] = source
    return grouped

# Агрегация данных по золотым слиткам
kyrgyz_gold_grouped = aggregate_gold_data(kyrgyz_gold_df, 'Kyrgyz Gold')
nbrk_grouped = aggregate_gold_data(nbrk_df, 'НБРК')

# Группировка данных Optima
optima_grouped = optima_data.groupby('Дата').agg({
    'Валюта': lambda x: ', '.join(map(str, set(x))),
    'Покупка': lambda x: ', '.join(map(str, set(x))),
    'Продажа': lambda x: ', '.join(map(str, set(x))),
    'Курс': lambda x: ', '.join(map(str, set(x))),
    'Банк': lambda x: ', '.join(map(str, set(x)))
}).reset_index()

# Объединение данных о золотых слитках
gold_combined = pd.concat([kyrgyz_gold_grouped, nbrk_grouped], ignore_index=True)

# Финальное объединение
result_df = pd.merge(gold_combined, optima_grouped, on='Дата', how='outer')

# Сортировка по дате в убывающем порядке
result_df = result_df.sort_values('Дата', ascending=False)

# Переименование столбцов для большей ясности
result_df.columns = [
    'Дата', 
    'Слитки', 
    'Цена обратного выкупа', 
    'Цена продажи', 
    'Источник',
    'Валюта', 
    'Покупка', 
    'Продажа', 
    'Курс', 
    'Банк'
]

# Заполнение NaN
result_df = result_df.fillna('Нет данных')

# Вывод результата
print(result_df.to_string(index=False))

FileNotFoundError: [Errno 2] No such file or directory: 'data/nbrk.json'

In [16]:
import pandas as pd
import json
import numpy as np

# Load Kyrgyz Gold data from JSON
with open('C:\\Users\\asus\\MyProjects\\BankProject\\data\\kyrgyz_gold.json', 'r', encoding='utf-8') as f:
    kyrgyz_gold_data = json.load(f)

# Load NBKR data from JSON
with open('C:\\Users\\asus\\MyProjects\\BankProject\\data\\nbkr.json', 'r', encoding='utf-8') as f:
    nbkr_data = json.load(f)

# Load Optima data from CSV
optima_data = pd.read_csv('C:\\Users\\asus\\MyProjects\\BankProject\\data\\optima.csv')

# Convert Kyrgyz Gold and NBKR data to DataFrames
kyrgyz_gold_df = pd.DataFrame(kyrgyz_gold_data)
nbkr_df = pd.DataFrame(nbkr_data)

# Ensure date is in consistent format
def standardize_date(df):
    # Convert 'Дата' to datetime if it's not already
    if not pd.api.types.is_datetime64_any_dtype(df['Дата']):
        df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%Y', errors='coerce')
    return df

kyrgyz_gold_df = standardize_date(kyrgyz_gold_df)
nbkr_df = standardize_date(nbkr_df)
optima_data['Дата'] = pd.to_datetime(optima_data['Дата'])

# Prepare the DataFrames with consistent column names
kyrgyz_gold_df['Source'] = 'Kyrgyz Gold'
nbkr_df['Source'] = 'NBKR'

# Add source columns to Optima data
optima_data['Source'] = 'Optima'

# Merge gold-related data
gold_merged_df = pd.concat([kyrgyz_gold_df, nbkr_df], ignore_index=True)

def pivot_optima_data(df):
    # List of currencies
    currencies = ['USD', 'KZT', 'EUR', 'RUB']
    transaction_types = ['Наличные', 'Безналичные']
    
    # Get unique dates
    unique_dates = df['Дата'].unique()
    
    # Prepare list to store pivoted data rows
    pivoted_rows = []
    
    # Create rows for each unique date
    for date in unique_dates:
        row = {'Дата': date, 'Source': 'Optima'}
        
        # Create columns for each currency and transaction type
        for currency in currencies:
            for transaction_type in transaction_types:
                # Filter data for specific currency and transaction type
                currency_data = df[(df['Валюта'] == currency) & 
                                   (df['Курс'] == transaction_type) & 
                                   (df['Дата'] == date)]
                
                # Buying rate
                if not currency_data.empty:
                    row[f'{transaction_type}_{currency}_Покупка'] = currency_data['Покупка'].iloc[0]
                    row[f'{transaction_type}_{currency}_Продажа'] = currency_data['Продажа'].iloc[0]
                else:
                    row[f'{transaction_type}_{currency}_Покупка'] = np.nan
                    row[f'{transaction_type}_{currency}_Продажа'] = np.nan
        
        pivoted_rows.append(row)
    
    # Convert to DataFrame
    pivoted_df = pd.DataFrame(pivoted_rows)
    
    return pivoted_df

# Pivot Optima data
optima_pivoted_df = pivot_optima_data(optima_data)

# Final merged DataFrame
final_df = pd.merge(
    gold_merged_df, 
    optima_pivoted_df, 
    on=['Дата', 'Source'], 
    how='outer'
)

# Sort by date
final_df = final_df.sort_values('Дата')

# Display the first few rows to verify
print(final_df.head())

# Optional: Save to CSV
final_df.to_csv('C:\\Users\\asus\\MyProjects\\BankProject\\merged_financial_data.csv', index=False, encoding='utf-8')

final_df

        Дата Мерные золотые слитки, г. Цена обратного выкупа, сом  \
0 2023-12-11                      1.00                   7 161.00   
1 2023-12-11                      2.00                  13 045.00   
2 2023-12-11                      5.00                  30 664.00   
3 2023-12-11                     10.00                  60 418.50   
4 2023-12-11                   31.1035                 185 047.00   

  Цена продажи, сом Source  Наличные_USD_Покупка  Наличные_USD_Продажа  \
0          7 196.50   NBKR                   NaN                   NaN   
1         13 097.00   NBKR                   NaN                   NaN   
2         30 756.00   NBKR                   NaN                   NaN   
3         60 539.50   NBKR                   NaN                   NaN   
4        187 822.50   NBKR                   NaN                   NaN   

   Безналичные_USD_Покупка  Безналичные_USD_Продажа  Наличные_KZT_Покупка  \
0                      NaN                      NaN            

,Дата,"Мерные золотые слитки, г.","Цена обратного выкупа, сом","Цена продажи, сом",Source,Наличные_USD_Покупка,Наличные_USD_Продажа,Безналичные_USD_Покупка,Безналичные_USD_Продажа,Наличные_KZT_Покупка,...,Безналичные_KZT_Покупка,Безналичные_KZT_Продажа,Наличные_EUR_Покупка,Наличные_EUR_Продажа,Безналичные_EUR_Покупка,Безналичные_EUR_Продажа,Наличные_RUB_Покупка,Наличные_RUB_Продажа,Безналичные_RUB_Покупка,Безналичные_RUB_Продажа
0,2023-12-11,1.00,7 161.00,7 196.50,NBKR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-12-11,2.00,13 045.00,13 097.00,NBKR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-12-11,5.00,30 664.00,30 756.00,NBKR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-12-11,10.00,60 418.50,60 539.50,NBKR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-12-11,31.1035,185 047.00,187 822.50,NBKR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042,2024-12-14,5,38 856.00,38 972.50,Kyrgyz Gold,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2040,2024-12-14,1,8 734.00,8 777.50,Kyrgyz Gold,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2049,2024-12-14,NaN,NaN,NaN,Optima,86.85,87.35,NaN,NaN,0.116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2044,2024-12-14,20,151 085.50,153 352.00,Kyrgyz Gold,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
import pandas as pd
import json
import numpy as np

# Load Kyrgyz Gold data from JSON
with open('C:\\Users\\asus\\MyProjects\\BankProject\\data\\kyrgyz_gold.json', 'r', encoding='utf-8') as f:
    kyrgyz_gold_data = json.load(f)

# Load NBKR data from JSON
with open('C:\\Users\\asus\\MyProjects\\BankProject\\data\\nbkr.json', 'r', encoding='utf-8') as f:
    nbkr_data = json.load(f)

# Load Optima data from CSV
optima_data = pd.read_csv('C:\\Users\\asus\\MyProjects\\BankProject\\data\\optima.csv')

# Convert Kyrgyz Gold and NBKR data to DataFrames
kyrgyz_gold_df = pd.DataFrame(kyrgyz_gold_data)
nbkr_df = pd.DataFrame(nbkr_data)

# Ensure consistent date formatting
def standardize_date(df, from_format='%d.%m.%Y'):
    # Convert 'Дата' to datetime 
    df['Дата'] = pd.to_datetime(df['Дата'], format=from_format, errors='coerce')
    return df

# Standardize dates for gold and NBKR data
kyrgyz_gold_df = standardize_date(kyrgyz_gold_df)
nbkr_df = standardize_date(nbkr_df)

# Optima data is already in datetime format
optima_data['Дата'] = pd.to_datetime(optima_data['Дата'])

# Prepare the DataFrames with consistent column names
kyrgyz_gold_df['Source'] = 'Kyrgyz Gold'
nbkr_df['Source'] = 'NBKR'
optima_data['Source'] = 'Optima'

# Merge gold-related data
gold_merged_df = pd.concat([kyrgyz_gold_df, nbkr_df], ignore_index=True)

def pivot_optima_data(df):
    # List of currencies
    currencies = ['USD', 'KZT', 'EUR', 'RUB']
    transaction_types = ['Наличные', 'Безналичные']
    
    # Prepare list to store pivoted data rows
    pivoted_rows = []
    
    # Group by date and transaction type
    grouped = df.groupby(['Дата', 'Курс'])
    
    # Get unique dates
    unique_dates = df['Дата'].unique()
    
    # Create rows for each unique date
    for date in unique_dates:
        row = {'Дата': date, 'Source': 'Optima'}
        
        # Create columns for each currency and transaction type
        for currency in currencies:
            for transaction_type in transaction_types:
                # Filter data for specific currency and transaction type
                currency_data = df[(df['Валюта'] == currency) & 
                                    (df['Курс'] == transaction_type) & 
                                    (df['Дата'] == date)]
                
                # Buying and Selling rates
                if not currency_data.empty:
                    row[f'{transaction_type}_{currency}_Покупка'] = currency_data['Покупка'].iloc[0]
                    row[f'{transaction_type}_{currency}_Продажа'] = currency_data['Продажа'].iloc[0]
                else:
                    row[f'{transaction_type}_{currency}_Покупка'] = np.nan
                    row[f'{transaction_type}_{currency}_Продажа'] = np.nan
        
        # Add the row to the list of pivoted rows
        pivoted_rows.append(row)
    
    # Convert list of rows to DataFrame
    pivoted_df = pd.DataFrame(pivoted_rows)
    
    return pivoted_df


# Pivot Optima data
optima_pivoted_df = pivot_optima_data(optima_data)

# Final merged DataFrame
final_df = pd.merge(
    gold_merged_df, 
    optima_pivoted_df, 
    on=['Дата', 'Source'], 
    how='outer'
)

final_df = final_df.reset_index(drop=True)
final_df = final_df.sort_values('Дата', ascending=False)
final_df = final_df.reset_index(drop=True)
print(final_df.head())

final_df.to_csv('C:\\Users\\asus\\MyProjects\\BankProject\\merged_financial_data_sorted.csv', index=False, encoding='utf-8')
final_df

        Дата Мерные золотые слитки, г. Цена обратного выкупа, сом  \
0 2024-12-17                       NaN                        NaN   
1 2024-12-17                       100                 745 879.50   
2 2024-12-17                         1                   8 724.00   
3 2024-12-17                         2                  16 250.00   
4 2024-12-17                        10                  75 957.50   

  Цена продажи, сом       Source  Наличные_USD_Покупка  Наличные_USD_Продажа  \
0               NaN       Optima                 86.85                 87.35   
1        768 256.00  Kyrgyz Gold                   NaN                   NaN   
2          8 767.50  Kyrgyz Gold                   NaN                   NaN   
3         16 315.00  Kyrgyz Gold                   NaN                   NaN   
4         76 109.00  Kyrgyz Gold                   NaN                   NaN   

   Безналичные_USD_Покупка  Безналичные_USD_Продажа  Наличные_KZT_Покупка  \
0                    86.65 

,Дата,"Мерные золотые слитки, г.","Цена обратного выкупа, сом","Цена продажи, сом",Source,Наличные_USD_Покупка,Наличные_USD_Продажа,Безналичные_USD_Покупка,Безналичные_USD_Продажа,Наличные_KZT_Покупка,...,Безналичные_KZT_Покупка,Безналичные_KZT_Продажа,Наличные_EUR_Покупка,Наличные_EUR_Продажа,Безналичные_EUR_Покупка,Безналичные_EUR_Продажа,Наличные_RUB_Покупка,Наличные_RUB_Продажа,Безналичные_RUB_Покупка,Безналичные_RUB_Продажа
0,2024-12-17,NaN,NaN,NaN,Optima,86.85,87.35,86.65,87.05,0.116,...,0.116,0.182,91.0,92.0,91.2,92.2,0.83,0.86,0.8,0.88
1,2024-12-17,100,745 879.50,768 256.00,Kyrgyz Gold,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-12-17,1,8 724.00,8 767.50,Kyrgyz Gold,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-12-17,2,16 250.00,16 315.00,Kyrgyz Gold,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-12-17,10,75 957.50,76 109.00,Kyrgyz Gold,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2068,2023-12-11,5.00,30 664.00,30 756.00,NBKR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2069,2023-12-11,2.00,13 045.00,13 097.00,NBKR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2070,2023-12-11,31.1035,185 047.00,187 822.50,NBKR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2071,2023-12-11,100.00,584 650.00,602 189.50,NBKR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
import pandas as pd
import json
import numpy as np
from pandas import json_normalize

# 1. Function to clean and convert price columns
def clean_price(price_str):
    if isinstance(price_str, str):
        return float(price_str.replace(' ', '').replace(',', '.'))
    return np.nan

# 2. Load and preprocess Kyrgyz Gold JSON data
with open('C:\\Users\\asus\\MyProjects\\BankProject\\data\\kyrgyz_gold.json', 'r', encoding='utf-8') as f:
    kyrgyz_gold_data = json.load(f)

# Convert to DataFrame
kyrgyz_gold_df = pd.DataFrame(kyrgyz_gold_data)

# Rename 'Мерные золотые слитки, г.' to 'Тип'
kyrgyz_gold_df.rename(columns={'Мерные золотые слитки, г.': 'Тип'}, inplace=True)

# Convert 'Тип' to float (since some values in nbkr.json have decimals)
kyrgyz_gold_df['Тип'] = kyrgyz_gold_df['Тип'].astype(float)

# Clean and convert price columns
kyrgyz_gold_df['Цена обратного выкупа, сом'] = kyrgyz_gold_df['Цена обратного выкупа, сом'].apply(clean_price)
kyrgyz_gold_df['Цена продажи, сом'] = kyrgyz_gold_df['Цена продажи, сом'].apply(clean_price)

# 3. Load and preprocess NBKR JSON data
with open('C:\\Users\\asus\\MyProjects\\BankProject\\data\\nbkr.json', 'r', encoding='utf-8') as f:
    nbkr_data = json.load(f)

# Convert to DataFrame
nbkr_df = pd.DataFrame(nbkr_data)

# Rename 'Мерные золотые слитки, г.' to 'Тип'
nbkr_df.rename(columns={'Мерные золотые слитки, г.': 'Тип'}, inplace=True)

# Convert 'Тип' to float
nbkr_df['Тип'] = nbkr_df['Тип'].astype(float)

# Clean and convert price columns
nbkr_df['Цена обратного выкупа, сом'] = nbkr_df['Цена обратного выкупа, сом'].apply(clean_price)
nbkr_df['Цена продажи, сом'] = nbkr_df['Цена продажи, сом'].apply(clean_price)

# 4. Load Optima data from CSV
optima_data = pd.read_csv('C:\\Users\\asus\\MyProjects\\BankProject\\data\\optima.csv')

# Convert 'Дата' to datetime
def standardize_date(df, date_column='Дата', from_format='%d.%m.%Y'):
    df[date_column] = pd.to_datetime(df[date_column], format=from_format, errors='coerce')
    return df

kyrgyz_gold_df = standardize_date(kyrgyz_gold_df)
nbkr_df = standardize_date(nbkr_df)
optima_data['Дата'] = pd.to_datetime(optima_data['Дата'], errors='coerce')

# 5. Define the transformation function
def transform_gold_data(df, source):
    """
    Transforms the gold data by creating separate columns for each type of slitok's buy and sell prices.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing gold data.
        source (str): The source identifier (e.g., 'Kyrgyz_Altyn', 'NBKR').

    Returns:
        pd.DataFrame: The transformed DataFrame with separate columns for each slitok type.
    """
    # Ensure required columns are present
    required_columns = ['Дата', 'Тип', 'Цена обратного выкупа, сом', 'Цена продажи, сом']
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Missing required column: {col}")

    # Initialize a DataFrame with unique dates
    df_transformed = df[['Дата']].drop_duplicates().copy()

    # Get unique types to handle all available types dynamically
    unique_types = df['Тип'].unique()
    unique_types = sorted(unique_types)  # Optional: sort types for consistency

    for size in unique_types:
        # Filter rows for the current type
        temp = df[df['Тип'] == size][['Дата', 'Цена обратного выкупа, сом', 'Цена продажи, сом']].copy()

        # Define new column names
        buy_col = f'{source}_Slitok_{int(size)}_Цена_обратного_выкупа'
        sell_col = f'{source}_Slitok_{int(size)}_Цена_продажи'

        # Rename columns
        temp.rename(columns={
            'Цена обратного выкупа, сом': buy_col,
            'Цена продажи, сом': sell_col
        }, inplace=True)

        # Merge with the transformed DataFrame
        df_transformed = pd.merge(df_transformed, temp, on='Дата', how='left')

    return df_transformed

# 6. Transform the data
kyrgyz_gold_transformed = transform_gold_data(kyrgyz_gold_df, 'Kyrgyz_Altyn')
nbkr_transformed = transform_gold_data(nbkr_df, 'NBKR')

# 7. Merge the transformed gold data
gold_merged_df = pd.concat([kyrgyz_gold_transformed, nbkr_transformed], axis=1)

# To avoid duplicate 'Дата' columns after concat, we'll merge on 'Дата'
gold_merged_df = pd.merge(kyrgyz_gold_transformed, nbkr_transformed, on='Дата', how='outer')

# 8. Preprocess Optima data
# Assuming 'Source' column exists and needs to be dropped
if 'Source' in optima_data.columns:
    optima_data.drop(columns=['Source'], inplace=True)

# 9. Merge all data on 'Дата'
final_df = pd.merge(
    gold_merged_df,
    optima_data,
    on='Дата',
    how='outer'
)

# 10. Sort by date descending and reset index
final_df = final_df.sort_values('Дата', ascending=False).reset_index(drop=True)

# 11. Display the first few rows for verification
print(final_df.head())

# 12. Save the final DataFrame to CSV
final_df.to_csv('C:\\Users\\asus\\MyProjects\\BankProject\\merged_financial_data_final.csv', index=False, encoding='utf-8-sig')  # Use 'utf-8-sig' for better compatibility with Excel
final_df

        Дата  Kyrgyz_Altyn_Slitok_1_Цена_обратного_выкупа  \
0 2024-12-17                                       8724.0   
1 2024-12-17                                       8724.0   
2 2024-12-17                                       8724.0   
3 2024-12-17                                       8724.0   
4 2024-12-17                                       8724.0   

   Kyrgyz_Altyn_Slitok_1_Цена_продажи  \
0                              8767.5   
1                              8767.5   
2                              8767.5   
3                              8767.5   
4                              8767.5   

   Kyrgyz_Altyn_Slitok_2_Цена_обратного_выкупа  \
0                                      16250.0   
1                                      16250.0   
2                                      16250.0   
3                                      16250.0   
4                                      16250.0   

   Kyrgyz_Altyn_Slitok_2_Цена_продажи  \
0                             16315.0   
1  

,Дата,Kyrgyz_Altyn_Slitok_1_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_1_Цена_продажи,Kyrgyz_Altyn_Slitok_2_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_2_Цена_продажи,Kyrgyz_Altyn_Slitok_5_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_5_Цена_продажи,Kyrgyz_Altyn_Slitok_10_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_10_Цена_продажи,Kyrgyz_Altyn_Slitok_20_Цена_обратного_выкупа,...,NBKR_Slitok_31_Цена_продажи,NBKR_Slitok_100_Цена_обратного_выкупа,NBKR_Slitok_100_Цена_продажи,Unnamed: 0,Валюта,Покупка,Продажа,Курс,Банк,Индекс
0,2024-12-17,8724.0,8767.5,16250.0,16315.0,38805.0,38921.5,75957.5,76109.0,150882.5,...,237116.5,746271.5,768659.5,NaN,RUB,0.800,0.880,Безналичные,Оптима,609.0
1,2024-12-17,8724.0,8767.5,16250.0,16315.0,38805.0,38921.5,75957.5,76109.0,150882.5,...,237116.5,746271.5,768659.5,NaN,KZT,0.116,0.182,Безналичные,Оптима,608.0
2,2024-12-17,8724.0,8767.5,16250.0,16315.0,38805.0,38921.5,75957.5,76109.0,150882.5,...,237116.5,746271.5,768659.5,NaN,EUR,91.200,92.200,Безналичные,Оптима,607.0
3,2024-12-17,8724.0,8767.5,16250.0,16315.0,38805.0,38921.5,75957.5,76109.0,150882.5,...,237116.5,746271.5,768659.5,NaN,USD,86.650,87.050,Безналичные,Оптима,606.0
4,2024-12-17,8724.0,8767.5,16250.0,16315.0,38805.0,38921.5,75957.5,76109.0,150882.5,...,237116.5,746271.5,768659.5,NaN,RUB,0.830,0.860,Наличные,Оптима,605.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,2023-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,190767.0,593977.0,611796.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
806,2023-12-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,185260.0,576534.0,593830.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807,2023-12-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,185352.0,576825.0,594129.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
808,2023-12-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,185878.0,578490.5,595845.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
import pandas as pd
import json
import numpy as np
from pandas import json_normalize

# 1. Function to clean and convert price columns
def clean_price(price_str):
    """
    Cleans and converts a price string to a float.
    
    Parameters:
        price_str (str): The price string with spaces and commas.
    
    Returns:
        float: The cleaned price as a float.
    """
    if isinstance(price_str, str):
        return float(price_str.replace(' ', '').replace(',', '.'))
    return np.nan

# 2. Load and preprocess Kyrgyz Gold JSON data
with open('C:\\Users\\asus\\MyProjects\\BankProject\\data\\kyrgyz_gold.json', 'r', encoding='utf-8') as f:
    kyrgyz_gold_data = json.load(f)

# Convert to DataFrame
kyrgyz_gold_df = pd.DataFrame(kyrgyz_gold_data)

# Rename 'Мерные золотые слитки, г.' to 'Тип'
kyrgyz_gold_df.rename(columns={'Мерные золотые слитки, г.': 'Тип'}, inplace=True)

# Convert 'Тип' to float (since some values in nbkr.json have decimals)
kyrgyz_gold_df['Тип'] = kyrgyz_gold_df['Тип'].astype(float)

# Clean and convert price columns
kyrgyz_gold_df['Цена обратного выкупа, сом'] = kyrgyz_gold_df['Цена обратного выкупа, сом'].apply(clean_price)
kyrgyz_gold_df['Цена продажи, сом'] = kyrgyz_gold_df['Цена продажи, сом'].apply(clean_price)

# 3. Load and preprocess NBKR JSON data
with open('C:\\Users\\asus\\MyProjects\\BankProject\\data\\nbkr.json', 'r', encoding='utf-8') as f:
    nbkr_data = json.load(f)

# Convert to DataFrame
nbkr_df = pd.DataFrame(nbkr_data)

# Rename 'Мерные золотые слитки, г.' to 'Тип'
nbkr_df.rename(columns={'Мерные золотые слитки, г.': 'Тип'}, inplace=True)

# Convert 'Тип' to float
nbkr_df['Тип'] = nbkr_df['Тип'].astype(float)

# Clean and convert price columns
nbkr_df['Цена обратного выкупа, сом'] = nbkr_df['Цена обратного выкупа, сом'].apply(clean_price)
nbkr_df['Цена продажи, сом'] = nbkr_df['Цена продажи, сом'].apply(clean_price)

# 4. Load Optima data from CSV
optima_data = pd.read_csv('C:\\Users\\asus\\MyProjects\\BankProject\\data\\optima.csv')

# 5. Convert 'Дата' to datetime
def standardize_date(df, date_column='Дата', from_format='%d.%m.%Y'):
    """
    Converts a date column to datetime format.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing the date column.
        date_column (str): The name of the date column.
        from_format (str): The format of the input date strings.
    
    Returns:
        pd.DataFrame: The DataFrame with the converted date column.
    """
    df[date_column] = pd.to_datetime(df[date_column], format=from_format, errors='coerce')
    return df

kyrgyz_gold_df = standardize_date(kyrgyz_gold_df)
nbkr_df = standardize_date(nbkr_df)
optima_data['Дата'] = pd.to_datetime(optima_data['Дата'], errors='coerce')

# 6. Define the transformation function for gold data
def transform_gold_data(df, source):
    """
    Transforms the gold data by creating separate columns for each type of slitok's buy and sell prices.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing gold data.
        source (str): The source identifier (e.g., 'Kyrgyz_Altyn', 'NBKR').

    Returns:
        pd.DataFrame: The transformed DataFrame with separate columns for each slitok type.
    """
    # Ensure required columns are present
    required_columns = ['Дата', 'Тип', 'Цена обратного выкупа, сом', 'Цена продажи, сом']
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Missing required column: {col}")

    # Initialize a DataFrame with unique dates
    df_transformed = df[['Дата']].drop_duplicates().copy()

    # Get unique types to handle all available types dynamically
    unique_types = df['Тип'].unique()
    unique_types = sorted(unique_types)  # Optional: sort types for consistency

    for size in unique_types:
        # Filter rows for the current type
        temp = df[df['Тип'] == size][['Дата', 'Цена обратного выкупа, сом', 'Цена продажи, сом']].copy()

        # Define new column names
        buy_col = f'{source}_Slitok_{int(size)}_Цена_обратного_выкупа'
        sell_col = f'{source}_Slitok_{int(size)}_Цена_продажи'

        # Rename columns
        temp.rename(columns={
            'Цена обратного выкупа, сом': buy_col,
            'Цена продажи, сом': sell_col
        }, inplace=True)

        # Merge with the transformed DataFrame
        df_transformed = pd.merge(df_transformed, temp, on='Дата', how='left')

    return df_transformed

# 7. Transform the gold data
kyrgyz_gold_transformed = transform_gold_data(kyrgyz_gold_df, 'Kyrgyz_Altyn')
nbkr_transformed = transform_gold_data(nbkr_df, 'NBKR')

# 8. Merge the transformed gold data
gold_merged_df = pd.merge(kyrgyz_gold_transformed, nbkr_transformed, on='Дата', how='outer')

# 9. Preprocess Optima data
# Assuming 'Source' column exists and needs to be dropped
if 'Source' in optima_data.columns:
    optima_data.drop(columns=['Source'], inplace=True)

# 10. Combine "Наличные" and "Безналичные" in Optima data
# **Clarification Needed:** The sample data provided does not include a column that explicitly indicates "Наличные" or "Безналичные".
# For the purpose of this guide, we'll assume that there's a column named 'Тип_Транзакции' that indicates the transaction type.
# If such a column does not exist, you'll need to adjust the code accordingly.

# **Assumption:** There's a column 'Тип_Транзакции' in `optima_data` indicating 'Наличные' or 'Безналичные'.
# If not, please provide more details on how to identify the transaction type.

# Check if 'Тип_Транзакции' exists
if 'Тип_Транзакции' not in optima_data.columns:
    # **Alternative Approach:** If 'Тип_Транзакции' is not present, and 'Курс' does not indicate transaction type,
    # we might need to create it based on other data or handle it differently.
    # For demonstration, let's assume all transactions are 'Безналичные' if not specified.
    optima_data['Тип_Транзакции'] = 'Безналичные'  # Default value; adjust as needed

# 11. Pivot the Optima data to include 'Валюта' as a feature
# This will create separate columns for each currency's purchase and sale rates, differentiated by transaction type.

# Pivot the Optima data
optima_pivot = optima_data.pivot_table(
    index='Дата',
    columns=['Валюта', 'Тип_Транзакции'],
    values=['Покупка', 'Продажа'],
    aggfunc='first'  # Assuming one entry per combination; adjust if necessary
)

# Flatten the MultiIndex columns
optima_pivot.columns = [f'Optima_{val}_{trans}' for val, trans in optima_pivot.columns]
optima_pivot.reset_index(inplace=True)

# 12. Merge the pivoted Optima data with gold data
final_df = pd.merge(
    gold_merged_df,
    optima_pivot,
    on='Дата',
    how='outer'
)

# 13. Sort by date descending and reset index
final_df = final_df.sort_values('Дата', ascending=False).reset_index(drop=True)

# 14. Display the first few rows for verification
print(final_df.head())

# 15. Save the final DataFrame to CSV
final_df.to_csv('C:\\Users\\asus\\MyProjects\\BankProject\\merged_financial_data_final.csv', index=False, encoding='utf-8-sig')  # Use 'utf-8-sig' for better compatibility with Excel
final_df

ValueError: too many values to unpack (expected 2)

In [31]:
import pandas as pd
import json
import numpy as np
from pandas import json_normalize

# 1. Function to clean and convert price columns
def clean_price(price_str):
    """
    Cleans and converts a price string to a float.
    
    Parameters:
        price_str (str): The price string with spaces and commas.
    
    Returns:
        float: The cleaned price as a float.
    """
    if isinstance(price_str, str):
        return float(price_str.replace(' ', '').replace(',', '.'))
    return np.nan

# 2. Load and preprocess Kyrgyz Gold JSON data
with open('C:\\Users\\asus\\MyProjects\\BankProject\\data\\kyrgyz_gold.json', 'r', encoding='utf-8') as f:
    kyrgyz_gold_data = json.load(f)

# Convert to DataFrame
kyrgyz_gold_df = pd.DataFrame(kyrgyz_gold_data)

# Rename 'Мерные золотые слитки, г.' to 'Тип'
kyrgyz_gold_df.rename(columns={'Мерные золотые слитки, г.': 'Тип'}, inplace=True)

# Convert 'Тип' to float (since some values in nbkr.json have decimals)
kyrgyz_gold_df['Тип'] = kyrgyz_gold_df['Тип'].astype(float)

# Clean and convert price columns
kyrgyz_gold_df['Цена обратного выкупа, сом'] = kyrgyz_gold_df['Цена обратного выкупа, сом'].apply(clean_price)
kyrgyz_gold_df['Цена продажи, сом'] = kyrgyz_gold_df['Цена продажи, сом'].apply(clean_price)

# 3. Load and preprocess NBKR JSON data
with open('C:\\Users\\asus\\MyProjects\\BankProject\\data\\nbkr.json', 'r', encoding='utf-8') as f:
    nbkr_data = json.load(f)

# Convert to DataFrame
nbkr_df = pd.DataFrame(nbkr_data)

# Rename 'Мерные золотые слитки, г.' to 'Тип'
nbkr_df.rename(columns={'Мерные золотые слитки, г.': 'Тип'}, inplace=True)

# Convert 'Тип' to float
nbkr_df['Тип'] = nbkr_df['Тип'].astype(float)

# Clean and convert price columns
nbkr_df['Цена обратного выкупа, сом'] = nbkr_df['Цена обратного выкупа, сом'].apply(clean_price)
nbkr_df['Цена продажи, сом'] = nbkr_df['Цена продажи, сом'].apply(clean_price)

# 4. Load Optima data from CSV
optima_data = pd.read_csv('C:\\Users\\asus\\MyProjects\\BankProject\\data\\optima.csv')

# 5. Convert 'Дата' to datetime
def standardize_date(df, date_column='Дата', from_format='%d.%m.%Y'):
    """
    Converts a date column to datetime format.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing the date column.
        date_column (str): The name of the date column.
        from_format (str): The format of the input date strings.
    
    Returns:
        pd.DataFrame: The DataFrame with the converted date column.
    """
    df[date_column] = pd.to_datetime(df[date_column], format=from_format, errors='coerce')
    return df

kyrgyz_gold_df = standardize_date(kyrgyz_gold_df)
nbkr_df = standardize_date(nbkr_df)
optima_data['Дата'] = pd.to_datetime(optima_data['Дата'], errors='coerce')

# 6. Define the transformation function for gold data
def transform_gold_data(df, source):
    """
    Transforms the gold data by creating separate columns for each type of slitok's buy and sell prices.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing gold data.
        source (str): The source identifier (e.g., 'Kyrgyz_Altyn', 'NBKR').

    Returns:
        pd.DataFrame: The transformed DataFrame with separate columns for each slitok type.
    """
    # Ensure required columns are present
    required_columns = ['Дата', 'Тип', 'Цена обратного выкупа, сом', 'Цена продажи, сом']
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Missing required column: {col}")

    # Initialize a DataFrame with unique dates
    df_transformed = df[['Дата']].drop_duplicates().copy()

    # Get unique types to handle all available types dynamically
    unique_types = df['Тип'].unique()
    unique_types = sorted(unique_types)  # Optional: sort types for consistency

    for size in unique_types:
        # Filter rows for the current type
        temp = df[df['Тип'] == size][['Дата', 'Цена обратного выкупа, сом', 'Цена продажи, сом']].copy()

        # Define new column names
        buy_col = f'{source}_Slitok_{int(size)}_Цена_обратного_выкупа'
        sell_col = f'{source}_Slitok_{int(size)}_Цена_продажи'

        # Rename columns
        temp.rename(columns={
            'Цена обратного выкупа, сом': buy_col,
            'Цена продажи, сом': sell_col
        }, inplace=True)

        # Merge with the transformed DataFrame
        df_transformed = pd.merge(df_transformed, temp, on='Дата', how='left')

    return df_transformed

# 7. Transform the gold data
kyrgyz_gold_transformed = transform_gold_data(kyrgyz_gold_df, 'Kyrgyz_Altyn')
nbkr_transformed = transform_gold_data(nbkr_df, 'NBKR')

# 8. Merge the transformed gold data
gold_merged_df = pd.merge(kyrgyz_gold_transformed, nbkr_transformed, on='Дата', how='outer')

# 9. Preprocess Optima data
# Assuming 'Source' column exists and needs to be dropped
if 'Source' in optima_data.columns:
    optima_data.drop(columns=['Source'], inplace=True)

# 10. Combine "Наличные" and "Безналичные" in Optima data
# **Clarification Needed:** The sample data provided does not include a column that explicitly indicates "Наличные" or "Безналичные".
# For the purpose of this guide, we'll assume that there's a column named 'Тип_Транзакции' that indicates the transaction type.
# If such a column does not exist, you'll need to adjust the code accordingly.

# **Assumption:** There's a column 'Тип_Транзакции' in `optima_data` indicating 'Наличные' or 'Безналичные'.
# If not, please provide more details on how to identify the transaction type.

# Check if 'Тип_Транзакции' exists
if 'Тип_Транзакции' not in optima_data.columns:
    # **Alternative Approach:** If 'Тип_Транзакции' is not present, and 'Курс' does not indicate transaction type,
    # we might need to create it based on other data or handle it differently.
    # For demonstration, let's assume all transactions are 'Безналичные' if not specified.
    optima_data['Тип_Транзакции'] = 'Безналичные'  # Default value; adjust as needed

# 11. Pivot the Optima data to include 'Валюта' as a feature
# This will create separate columns for each currency's purchase and sale rates, differentiated by transaction type.

# Pivot the Optima data
optima_pivot = optima_data.pivot_table(
    index='Дата',
    columns=['Валюта', 'Тип_Транзакции'],
    values=['Покупка', 'Продажа'],
    aggfunc='first'  # Assuming one entry per combination; adjust if necessary
)

# **Fix for MultiIndex with Three Levels: Flatten the columns appropriately**

# Option 1: Unpack all three levels
# optima_pivot.columns = [
#     f'Optima_{measure}_{currency}_{trans}'
#     for measure, currency, trans in optima_pivot.columns
# ]

# Option 2: General Flattening Using `join` (Recommended for flexibility)
optima_pivot.columns = [
    'Optima_' + '_'.join(map(str, col)) 
    for col in optima_pivot.columns
]

optima_pivot.reset_index(inplace=True)

# 12. Merge the pivoted Optima data with gold data
final_df = pd.merge(
    gold_merged_df,
    optima_pivot,
    on='Дата',
    how='outer'
)

# 13. Sort by date descending and reset index
final_df = final_df.sort_values('Дата', ascending=False).reset_index(drop=True)

# 14. Display the first few rows for verification
print(final_df.head())

# 15. Save the final DataFrame to CSV
final_df.to_csv('C:\\Users\\asus\\MyProjects\\BankProject\\merged_financial_data_final.csv', index=False, encoding='utf-8-sig')  # Use 'utf-8-sig' for better compatibility with Excel
final_df

        Дата  Kyrgyz_Altyn_Slitok_1_Цена_обратного_выкупа  \
0 2024-12-17                                       8724.0   
1 2024-12-16                                          NaN   
2 2024-12-15                                          NaN   
3 2024-12-14                                       8734.0   
4 2024-12-13                                       8791.0   

   Kyrgyz_Altyn_Slitok_1_Цена_продажи  \
0                              8767.5   
1                                 NaN   
2                                 NaN   
3                              8777.5   
4                              8835.0   

   Kyrgyz_Altyn_Slitok_2_Цена_обратного_выкупа  \
0                                      16250.0   
1                                          NaN   
2                                          NaN   
3                                      16270.0   
4                                      16383.5   

   Kyrgyz_Altyn_Slitok_2_Цена_продажи  \
0                             16315.0   
1  

,Дата,Kyrgyz_Altyn_Slitok_1_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_1_Цена_продажи,Kyrgyz_Altyn_Slitok_2_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_2_Цена_продажи,Kyrgyz_Altyn_Slitok_5_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_5_Цена_продажи,Kyrgyz_Altyn_Slitok_10_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_10_Цена_продажи,Kyrgyz_Altyn_Slitok_20_Цена_обратного_выкупа,...,NBKR_Slitok_100_Цена_обратного_выкупа,NBKR_Slitok_100_Цена_продажи,Optima_Покупка_EUR_Безналичные,Optima_Покупка_KZT_Безналичные,Optima_Покупка_RUB_Безналичные,Optima_Покупка_USD_Безналичные,Optima_Продажа_EUR_Безналичные,Optima_Продажа_KZT_Безналичные,Optima_Продажа_RUB_Безналичные,Optima_Продажа_USD_Безналичные
0,2024-12-17,8724.0,8767.5,16250.0,16315.0,38805.0,38921.5,75957.5,76109.0,150882.5,...,746271.5,768659.5,91.0,0.116,0.830,86.85,92.0,0.182,0.860,87.35
1,2024-12-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,747286.0,769704.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,90.7,0.116,0.825,86.85,91.7,0.182,0.855,87.35
3,2024-12-14,8734.0,8777.5,16270.0,16335.0,38856.0,38972.5,76059.0,76211.0,151085.5,...,NaN,NaN,NaN,0.116,NaN,86.85,NaN,0.182,NaN,87.35
4,2024-12-13,8791.0,8835.0,16383.5,16449.0,39142.0,39259.5,76631.0,76784.5,152228.0,...,752998.0,775588.0,90.7,0.116,0.825,86.85,91.7,0.182,0.855,87.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,2023-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,593977.0,611796.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273,2023-12-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,576534.0,593830.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,2023-12-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,576825.0,594129.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275,2023-12-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,578490.5,595845.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# 13. **Фильтрация данных: удаление записей с датами ранее 1 октября 2024 года**
final_df = final_df[final_df['Дата'] >= '2024-10-01']

# 14. Сортировка по дате в порядке убывания и сброс индекса
final_df = final_df.sort_values('Дата', ascending=False).reset_index(drop=True)

final_df

,Дата,Kyrgyz_Altyn_Slitok_1_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_1_Цена_продажи,Kyrgyz_Altyn_Slitok_2_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_2_Цена_продажи,Kyrgyz_Altyn_Slitok_5_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_5_Цена_продажи,Kyrgyz_Altyn_Slitok_10_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_10_Цена_продажи,Kyrgyz_Altyn_Slitok_20_Цена_обратного_выкупа,...,NBKR_Slitok_100_Цена_обратного_выкупа,NBKR_Slitok_100_Цена_продажи,Optima_Покупка_EUR_Безналичные,Optima_Покупка_KZT_Безналичные,Optima_Покупка_RUB_Безналичные,Optima_Покупка_USD_Безналичные,Optima_Продажа_EUR_Безналичные,Optima_Продажа_KZT_Безналичные,Optima_Продажа_RUB_Безналичные,Optima_Продажа_USD_Безналичные
0,2024-12-17,8724.0,8767.5,16250.0,16315.0,38805.0,38921.5,75957.5,76109.0,150882.5,...,746271.5,768659.5,91.0,0.116,0.830,86.85,92.0,0.182,0.860,87.35
1,2024-12-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,747286.0,769704.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,90.7,0.116,0.825,86.85,91.7,0.182,0.855,87.35
3,2024-12-14,8734.0,8777.5,16270.0,16335.0,38856.0,38972.5,76059.0,76211.0,151085.5,...,NaN,NaN,NaN,0.116,NaN,86.85,NaN,0.182,NaN,87.35
4,2024-12-13,8791.0,8835.0,16383.5,16449.0,39142.0,39259.5,76631.0,76784.5,152228.0,...,752998.0,775588.0,90.7,0.116,0.825,86.85,91.7,0.182,0.855,87.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,2024-10-05,8489.0,8531.5,15814.0,15877.0,37732.5,37845.5,73847.0,73994.5,146807.0,...,NaN,NaN,90.7,0.116,0.825,86.85,91.7,0.182,0.855,87.35
74,2024-10-04,8462.0,8504.5,15760.0,15823.0,37597.5,37710.5,73577.0,73724.5,146267.5,...,722838.0,744523.0,90.7,0.116,0.825,86.85,91.7,0.182,0.855,87.35
75,2024-10-03,8491.0,8533.5,15819.0,15882.5,37750.5,37864.0,73885.5,74033.0,146885.0,...,725936.0,747714.0,90.7,0.116,0.825,86.85,91.7,0.182,0.855,87.35
76,2024-10-02,8495.5,8538.0,15828.0,15891.0,37772.0,37885.5,73928.5,74076.0,146971.0,...,726366.0,748157.0,90.7,0.116,0.825,86.85,91.7,0.182,0.855,87.35


In [34]:
final_df.to_csv('data.csv')

In [33]:
final_df.columns

Index(['Дата', 'Kyrgyz_Altyn_Slitok_1_Цена_обратного_выкупа',
       'Kyrgyz_Altyn_Slitok_1_Цена_продажи',
       'Kyrgyz_Altyn_Slitok_2_Цена_обратного_выкупа',
       'Kyrgyz_Altyn_Slitok_2_Цена_продажи',
       'Kyrgyz_Altyn_Slitok_5_Цена_обратного_выкупа',
       'Kyrgyz_Altyn_Slitok_5_Цена_продажи',
       'Kyrgyz_Altyn_Slitok_10_Цена_обратного_выкупа',
       'Kyrgyz_Altyn_Slitok_10_Цена_продажи',
       'Kyrgyz_Altyn_Slitok_20_Цена_обратного_выкупа',
       'Kyrgyz_Altyn_Slitok_20_Цена_продажи',
       'Kyrgyz_Altyn_Slitok_31_Цена_обратного_выкупа',
       'Kyrgyz_Altyn_Slitok_31_Цена_продажи',
       'Kyrgyz_Altyn_Slitok_50_Цена_обратного_выкупа',
       'Kyrgyz_Altyn_Slitok_50_Цена_продажи',
       'Kyrgyz_Altyn_Slitok_100_Цена_обратного_выкупа',
       'Kyrgyz_Altyn_Slitok_100_Цена_продажи',
       'Kyrgyz_Altyn_Slitok_1000_Цена_обратного_выкупа',
       'Kyrgyz_Altyn_Slitok_1000_Цена_продажи',
       'NBKR_Slitok_1_Цена_обратного_выкупа', 'NBKR_Slitok_1_Цена_продажи',
       

In [39]:
import pandas as pd

# Assuming you have your data in a CSV or similar file format
# Load the data into a Pandas DataFrame
df = pd.read_csv('data.csv')

# Check the structure of your data
print(df.head())

# Convert the 'Дата' column to datetime format if it's not already
df['Дата'] = pd.to_datetime(df['Дата'], format='%Y-%m-%d')

# Calculate some financial statistics (example: calculating the difference between 'Цена_обратного_выкупа' and 'Цена_продажи')
df['Цена_разница'] = df['Kyrgyz_Altyn_Slitok_1_Цена_обратного_выкупа'] - df['Kyrgyz_Altyn_Slitok_1_Цена_продажи']

# If you want to filter rows where a certain value condition is met (e.g., if the difference is greater than 100)
filtered_df = df[df['Цена_разница'] > 100]

# Handling missing values if necessary (fill with a specific value or drop)
df = df.fillna(method='ffill')  # Forward fill missing values

# Or you can drop rows with missing values
# df = df.dropna()

# You can also calculate summary statistics for specific columns, such as mean or median
mean_price = df['Kyrgyz_Altyn_Slitok_1_Цена_обратного_выкупа'].mean()
median_price = df['Kyrgyz_Altyn_Slitok_1_Цена_обратного_выкупа'].median()

# Print some statistics
print(f"Mean Price: {mean_price}")
print(f"Median Price: {median_price}")

# Save the modified data if needed
df.to_csv('cleaned_data.csv', index=False)

# Example of filtering data for a specific date range
start_date = '2024-12-01'
end_date = '2024-12-10'
filtered_date_df = df[(df['Дата'] >= start_date) & (df['Дата'] <= end_date)]

# Print out filtered data for the date range
print(filtered_date_df)


   Unnamed: 0        Дата  Kyrgyz_Altyn_Slitok_1_Цена_обратного_выкупа  \
0           0  2024-12-17                                       8724.0   
1           1  2024-12-16                                          NaN   
2           2  2024-12-15                                          NaN   
3           3  2024-12-14                                       8734.0   
4           4  2024-12-13                                       8791.0   

   Kyrgyz_Altyn_Slitok_1_Цена_продажи  \
0                              8767.5   
1                                 NaN   
2                                 NaN   
3                              8777.5   
4                              8835.0   

   Kyrgyz_Altyn_Slitok_2_Цена_обратного_выкупа  \
0                                      16250.0   
1                                          NaN   
2                                          NaN   
3                                      16270.0   
4                                      16383.5   

   Kyrg

C:\Users\asus\AppData\Local\Temp\ipykernel_44676\784577752.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')  # Forward fill missing values


In [43]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import joblib
from datetime import timedelta

st.set_page_config(page_title="Optima Продажа EUR Безналичные", layout="wide")

def load_and_prepare_data(filepath):
    try:
        df = pd.read_csv(filepath, delimiter='\t')  # Adjust delimiter if necessary
        print(df.columns)  # Debugging line to check column names
        if 'Дата' not in df.columns:
            st.error("Column 'Дата' is missing from the dataset!")
            return None
        df['Дата'] = pd.to_datetime(df['Дата'], format='%Y-%m-%d')
        df = df.sort_values('Дата')
        df.set_index('Дата', inplace=True)
        df.fillna(method='ffill', inplace=True)  # Fill missing values with forward fill
        return df
    except Exception as e:
        st.error(f"Error loading data: {e}")
        return None


# Функция для подготовки признаков и целевой переменной
def get_features_and_target(df, target_column='Optima_Продажа_EUR_Безналичные'):
    df['Target'] = df[target_column].shift(-1)  # Предсказание следующего значения
    df = df.dropna()  # Удаляем строку с NaN в целевой переменной
    X = df.drop(['Target'], axis=1)
    y = df['Target']
    return X, y

# Функция для обучения модели
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Mean Absolute Error: {mae}')
    joblib.dump(model, 'optima_model.joblib')
    return model
# Main function
def main():
    st.title("Ежедневное предсказание Optima Продажа EUR Безналичные")

    # Загружаем данные
    DATA_PATH = 'cleaned_data.csv'  # Path to your file
    df = load_and_prepare_data(DATA_PATH)

    # Check if df is None
    if df is None:
        return  # Exit the function if there's an error loading data

    # Показываем последние 5 строк данных
    st.subheader("Исходные данные")
    st.write(df.tail())  # This will work now if df is valid

    # Подготовка признаков и целевой переменной
    target_column = 'Optima_Продажа_EUR_Безналичные'
    X, y = get_features_and_target(df, target_column)

    # Обучение модели (кэшируем для ускорения)
    @st.cache(allow_output_mutation=True)
    def get_model():
        try:
            model = joblib.load('optima_model.joblib')
        except:
            model = train_model(X, y)
        return model

    model = get_model()

    # Предсказание для следующего дня
    last_data = X.tail(1)
    prediction = model.predict(last_data)[0]
    
    st.subheader("Предсказание на следующий день")
    st.write(f"Предсказанное значение {target_column} на {df.index[-1] + timedelta(days=1)}: {prediction:.2f}")

    # Визуализация графика
    st.subheader("График Optima Продажа EUR Безналичные")
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(df.index, df[target_column], label='Фактические значения')
    ax.plot(df.index[-1] + timedelta(days=1), prediction, 'ro', label='Предсказание')
    ax.set_xlabel('Дата')
    ax.set_ylabel(target_column)
    ax.legend()
    st.pyplot(fig)

    # Дополнительные графики (предсказания на несколько шагов вперёд)
    st.subheader("История предсказаний")
    future_steps = 7
    future_predictions = []
    current_data = X.copy()

    for _ in range(future_steps):
        last_row = current_data.tail(1)
        pred = model.predict(last_row)[0]
        future_predictions.append(pred)
        new_row = last_row.copy()
        new_row[target_column] = pred
        new_row = new_row.drop('Target')
        current_data = current_data.append(new_row, ignore_index=True)

    future_dates = [df.index[-1] + timedelta(days=i+1) for i in range(future_steps)]

    fig2, ax2 = plt.subplots(figsize=(10, 5))
    ax2.plot(df.index, df[target_column], label='Фактические значения')
    ax2.plot(future_dates, future_predictions, 'ro-', label='Предсказания')
    ax2.set_xlabel('Дата')
    ax2.set_ylabel(target_column)
    ax2.legend()
    st.pyplot(fig2)

if __name__ == '__main__':
    main()

2024-12-17 15:39:45.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 15:39:45.362 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 15:39:45.362 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 15:39:45.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 15:39:45.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Index(['Unnamed: 0,Дата,Kyrgyz_Altyn_Slitok_1_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_1_Цена_продажи,Kyrgyz_Altyn_Slitok_2_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_2_Цена_продажи,Kyrgyz_Altyn_Slitok_5_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_5_Цена_продажи,Kyrgyz_Altyn_Slitok_10_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_10_Цена_продажи,Kyrgyz_Altyn_Slitok_20_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_20_Цена_продажи,Kyrgyz_Altyn_Slitok_31_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_31_Цена_продажи,Kyrgyz_Altyn_Slitok_50_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_50_Цена_продажи,Kyrgyz_Altyn_Slitok_100_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_100_Цена_продажи,Kyrgyz_Altyn_Slitok_1000_Цена_обратного_выкупа,Kyrgyz_Altyn_Slitok_1000_Цена_продажи,NBKR_Slitok_1_Цена_обратного_выкупа,NBKR_Slitok_1_Цена_продажи,NBKR_Slitok_2_Цена_обратного_выкупа,NBKR_Slitok_2_Цена_продажи,NBKR_Slitok_5_Цена_обратного_выкупа,NBKR_Slitok_5_Цена_продажи,NBKR_Slitok_10_Цена_обратного_выкупа,NBKR_Slitok_10_Цена_п